In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
import skimage
print(tf.VERSION)

conv = tf.layers.conv2d
bn = tf.layers.batch_normalization
relu = tf.nn.relu
he = tf.keras.initializers.he_normal
l2 = tf.contrib.layers.l2_regularizer

1.8.0


In [11]:
def id_block(inputs,lr,filters,training):
    conv1 = conv(inputs=inputs,filters=filters,kernel_size=[3,3],padding='same',activation=None,kernel_initializer=he,kernel_regularizer=l2(lr),name='conv1')
    bn1 = bn(inputs=conv1,training=training,name='bn1')
    relu1 = tf.nn.relu(bn1,name='relu1')
    conv2 = conv(inputs=relu1,filters=filters,kernel_size=[3,3],padding='same',activation=None,kernel_initializer=he,kernel_regularizer=l2(lr),name='conv2')
    bn2 = bn(inputs=conv2,training=training,name='bn2')
    skip = tf.add(inputs,bn2,name='skip')
    result = tf.nn.relu(skip,name='relu2')
    return result
    

In [12]:
def conv_block(inputs,lr,filters,training):
    conv_skip = conv(inputs=inputs,filters=filters,kernel_size=[2,2],strides=[2,2],padding='valid',activation=None,kernel_initializer=he,kernel_regularizer=l2(lr),name='conv-skip')
    bn_skip = bn(inputs=conv_skip,training=training,name='bn_skip')
    conv1 = conv(inputs=inputs,filters=filters,kernel_size=[3,3],strides=[2,2],padding='valid',activation=None,kernel_initializer=he,kernel_regularizer=l2(lr),name='conv1')
    bn1 = bn(inputs=conv1,training=training,name='bn1')
    relu1 = tf.nn.relu(bn1,name='relu1')
    conv2 = conv(inputs=relu1,filters=filters,kernel_size=[3,3],padding='same',activation=None,kernel_initializer=he,kernel_regularizer=l2(lr),name='conv2')
    bn2 = bn(inputs=conv2,training=training,name='bn2')
    skip_join = tf.add(bn_skip,bn2,name='skip_join')
    result = tf.nn.relu(skip_join,name='relu2')
    return result

In [13]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.cifar10.load_data()
x_train =  np.asarray(x_train,dtype=np.float32)
x_test =  np.asarray(x_test,dtype=np.float32)
y_train =  np.asarray(y_train,dtype=np.int32)
y_test =  np.asarray(y_test,dtype=np.int32)
#x_train /= 255.0;
#x_test /= 255.0;
mean = np.mean(x_train)
std = np.std(x_train)
x_train = (x_train - mean)/(std + 10e-7)
x_test = (x_test - mean)/(std + 10e-7)

feature_cols = [tf.feature_column.numeric_column('x')]

In [14]:
def augment(images, labels,resize=None,horizontal_flip=False,vertical_flip=False,rotate=0, crop_probability=0,crop_min_percent=0.6,crop_max_percent=1.,mixup=0):  
    if resize is not None:
        images = tf.image.resize_bilinear(images, resize)
  
  # My experiments showed that casting on GPU improves training performance
    if images.dtype != tf.float32:
        images = tf.image.convert_image_dtype(images, dtype=tf.float32)
        images = tf.subtract(images, 0.5)
        images = tf.multiply(images, 2.0)
        labels = tf.to_float(labels)

    with tf.name_scope('augmentation'):
        shp = tf.shape(images)
        batch_size, height, width = shp[0], shp[1], shp[2]
        width = tf.cast(width, tf.float32)
        height = tf.cast(height, tf.float32)
        transforms = []
        identity = tf.constant([1, 0, 0, 0, 1, 0, 0, 0], dtype=tf.float32)
        if horizontal_flip:
            coin = tf.less(tf.random_uniform([batch_size], 0, 1.0), 0.5)
            flip_transform = tf.convert_to_tensor(
          [-1., 0., width, 0., 1., 0., 0., 0.], dtype=tf.float32)
            transforms.append(
                tf.where(coin,
                   tf.tile(tf.expand_dims(flip_transform, 0), [batch_size, 1]),
                   tf.tile(tf.expand_dims(identity, 0), [batch_size, 1])))
       
        if vertical_flip:
            coin = tf.less(tf.random_uniform([batch_size], 0, 1.0), 0.5)
            flip_transform = tf.convert_to_tensor(
                [1, 0, 0, 0, -1, height, 0, 0], dtype=tf.float32)
            transforms.append(
                  tf.where(coin,
                   tf.tile(tf.expand_dims(flip_transform, 0), [batch_size, 1]),
                   tf.tile(tf.expand_dims(identity, 0), [batch_size, 1])))

        if rotate > 0:
            angle_rad = rotate / 180 * np.pi
            angles = tf.random_uniform([batch_size], -angle_rad, angle_rad)
            transforms.append(
            tf.contrib.image.angles_to_projective_transforms(
              angles, height, width))

        if crop_probability > 0:
            crop_pct = tf.random_uniform([batch_size], crop_min_percent,
                                   crop_max_percent)
            left = tf.random_uniform([batch_size], 0, width * (1 - crop_pct))
            top = tf.random_uniform([batch_size], 0, height * (1 - crop_pct))
            crop_transform = tf.stack([
                crop_pct,
                tf.zeros([batch_size]), top,
                tf.zeros([batch_size]), crop_pct, left,
                tf.zeros([batch_size]),tf.zeros([batch_size])], 1)

            coin = tf.less(
              tf.random_uniform([batch_size], 0, 1.0), crop_probability)
            transforms.append(
              tf.where(coin, crop_transform,
                   tf.tile(tf.expand_dims(identity, 0), [batch_size, 1])))

        if transforms:
              images = tf.contrib.image.transform(
              images,
              tf.contrib.image.compose_transforms(*transforms),
              interpolation='BILINEAR') # or 'NEAREST'

        def cshift(values): # Circular shift in batch dimension
              return tf.concat([values[-1:, ...], values[:-1, ...]], 0)

        if mixup > 0:
            mixup = 1.0 * mixup
            beta = tf.distributions.Beta(mixup, mixup)
            lam = beta.sample(batch_size)
            ll = tf.expand_dims(tf.expand_dims(tf.expand_dims(lam, -1), -1), -1)
            images = ll * images + (1 - ll) * cshift(images)
            labels = lam * labels + (1 - lam) * cshift(labels)

    return images, labels

In [15]:
def training_data(images,labels,batch_size,num):
    dataset = tf.data.Dataset.from_tensor_slices(({'x' : images},labels))
    def _aug(images,labels):
        images,labels = augment(images['x'],labels,horizontal_flip=True,rotate=15,crop_probability=0.8)
        return {'x' : images},labels
    augmented = dataset.map(_aug)
    dataset.concatenate(augmented)
    dataset = dataset.repeat().batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()
    

In [16]:
def resnet_model(features,labels,mode,params):
    lmd = params['regulariser']
    inputs = tf.feature_column.input_layer(features,params['feature_cols'])
    inputs = tf.reshape(inputs,[-1,32,32,3])
    inputs = conv(inputs=inputs,filters=16,kernel_size=[3,3],padding='same',activation=None,kernel_initializer=he,kernel_regularizer=l2(lmd),name='first_conv')
    inputs = bn(inputs=inputs,training=(mode==tf.estimator.ModeKeys.TRAIN),name='first_bn')
    inputs = tf.nn.relu(inputs,name='first_relu')
    
    for block,filters,num in params['resnet']:
        for i in range(num):
            with tf.variable_scope(block + str(filters) + "-" + str(i+1)):
                if block == 'id':
                    inputs = id_block(inputs,lmd,filters,mode==tf.estimator.ModeKeys.TRAIN)
                else:
                    inputs = conv_block(inputs,lmd,filters,mode==tf.estimator.ModeKeys.TRAIN)
        print("Dimensions now " + str(inputs.shape))
    
    inputs = tf.reduce_mean(inputs,axis=[1,2],name='global_avg_pool')
    print("Average pooling dimensions" + str(inputs.shape))
    inputs = tf.reshape(inputs,[-1,params['flat_size']])
    logits = tf.layers.dense(inputs=inputs,units=params['n_classes'],activation=tf.nn.relu,kernel_initializer=he,kernel_regularizer=l2(lmd))
    
    
    
    predictions = {
        'classes' : tf.argmax(logits,axis=1),
        'probabilities' : tf.nn.softmax(logits)
    }
    
    eval_ops = {
        'accuracy' : tf.metrics.accuracy(labels=labels,predictions=predictions['classes'])
    }
    
    tf.summary.scalar(eval_ops['accuracy'])
    
    if mode==tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(predictions=predictions,mode=mode)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,logits=logits)
    
    if mode==tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op)
    
    return tf.estimator.EstimatorSpec(mode=mode,loss=loss,eval_metric_ops=eval_ops)


In [ ]:
resnet56 = [('id',16,9),('conv',32,1),('id',32,9),('conv',64,1),('id',64,1)]
classifier = tf.estimator.Estimator(model_fn=resnet_model,model_dir='RESNET',params={
    'feature_cols' : feature_cols,
    'regulariser' : 0.0005,
    'learning_rate' : 0.001,
    'resnet' : resnet56,
    'flat_size' : 64
}) 
classifier.train(input_fn=lambda:training_data(x_train,y_train,250,x_train.shape[0]))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'RESNET', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efbe04a8a90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
